# Getting started with Lightdash

The Lightdash Python client allows you query data from the Lightdash Semantic Layer directly from Python. For example, you can use it to query your governed metrics for:
- Creating data visualisations and interactive apps (matplotlib, plotly, etc)
- Exploring data in a notebook (hex, jupyter, etc)
- Pulling data into orchestration pipelines (dagster, airflow, etc)

## Installation

```bash
pip install lightdash
```

## Setup Lightdash Client

Make sure you have the following environment variables set:
- `LIGHTDASH_ACCESS_TOKEN`
- `LIGHTDASH_PROJECT_UUID`
- `LIGHTDASH_INSTANCE_URL`

Then you can create a Lightdash client:

In [1]:
from lightdash import Client

import os
access_token = os.getenv("LIGHTDASH_ACCESS_TOKEN")
project_uuid = os.getenv("LIGHTDASH_PROJECT_UUID")
instance_url = os.getenv("LIGHTDASH_INSTANCE_URL")

client = Client(
    instance_url=instance_url,
    access_token=access_token,
    project_uuid=project_uuid
)

## Run a query

### 1. Start with a model

You can use the client to list available models in your semantic layer:

In [2]:
# print all models
client.list_models()

# get a specific model (try tab completion!)
orders = client.models.dbt_orders

In [3]:
client.list_models()

[Model(dbt_users: This table contains information on all of our users and customers),
 Model(dbt_baskets: This table contains all the information on customer shopping baskets on our site. There is one basket per order),
 Model(dbt_support_requests: This table contains information on all the support requests we’ve received),
 Model(dbt_orders: This table contains information on all the confirmed orders and their status.),
 Model(website_sessions),
 Model(virtual_github),
 Model(partner_orders)]

### 2. Find available metrics

Find metrics in your semantic layer so you can pull data from your warehouse without writing SQL

In [4]:
# Show metrics
orders.list_metrics()

# Get data about a metric
orders.metrics.sum_of_profit.description

'Sums the profit on all orders in your query, grouped by the dimensions you have chosen.'

In [5]:
orders.list_metrics()

[Metric(sum_of_profit: Sums the profit on all orders in your query, grouped by the dimensions you have chosen.),
 Metric(sum_of_basket_total: Sums the basket total for orders in your query, grouped by the dimensions you have chosen.),
 Metric(count_distinct_user_id: Counts the number of unique users (customers) with orders based on their User ID.),
 Metric(average_of_basket_total: Takes the average of basket total for orders in your query, grouped by the dimensions you have chosen.),
 Metric(count_distinct_order_id: This metric counts unique orders based on the Order ID field from the Orders table.)]

### 3. Run a query

Run a query to fetch metric data from your warehouse

In [6]:
orders.query(
    metrics=[orders.metrics.sum_of_profit]
).to_records()

[{'Sum of profit': 1515265.3899999303}]

Use dimensions to break down your metric data

In [5]:
# List available dimensions
orders.list_dimensions()

# Query profit broken down by partner
orders.query(
    dimensions=[orders.dimensions.partner_name],
    metrics=[orders.metrics.sum_of_profit]
).to_df()

,Partner name,Sum of profit
0,Happy Harvesters,83031.18
1,Garden of Sweden,52685.50
2,"Peat, Fruits and Leaves",120270.90
3,Redwood Ranch,708107.40
4,Potted Paradise,65746.48
5,Roots & Shoots,133449.58
6,Trowels R Us,94590.56
7,Sprout Society,114499.07
8,Plant Paradise,142884.72
